# Learning to Rank

The main goal of this assignment was to implement and compare the performance of several approaches to the Learning to Rank (LTR) problem. We received an implementation of a pointwise method base on squared-loss minimization and implemented RankNet (pairwise) and LambdaNet (listwise) algorithms.

## Methodology

The training of the methods was executed on a database with 5 cross-validation folds, each containing the 90 training, 30 validation and 30 test examples with no overlap. The dataset is constructed fora homepage finding task, therefore each query should have exactly one relevant document label. Initial data exploration indicated that some queries were noisy as they had zero or more than one relevant document labels. We took these cases into consideration in our implementation.

For each algorithm, we obtained results on all 5 folds of the data where we trained a model on the training data for 200 epochs and reported the objective function evaluation, training mNDCG, and validation mNDCG for each epoch. Across all folds of the dataset, we obtained five candidate models for each algorithm. For each algorithm we then chose a top candidate according to its validation mNDCG on the final training epoch. We used the top candidate to report the test mNDCG on all test data (obtained from the five folds).

We used L1 and L2 regularization using the default assignment values.

## Pointwise MSE

As our dataset is heavily unbalanced, with approximately one positive label per query, we did not expect to achieve reasonable mNDCG results from this algorithm. This is due to the fact that a point-wise approach does not take the ranking structure into account, and the training involves a squared-loss function instead of an IR metric directly.

![pointwise_train_loss](img/pointwise_train_loss.png)
<center> <b> Figure 1. Training Loss  </b> </center> 

![pointwise_train_mndcg](img/pointwise_train_mndcg.png)

![pointwise_val_mndcg](img/pointwise_val_mndcg.png)


## Pairwise RankNet

Plots, results, comments

We took advantage of the sparsity of the $S$ matrix used in RankNet and LambdaNet to improve the runtimes of our algorithms. We computed the $S$ matrix for each query in the training and validation set for each fold in advance, to reduce overhead. Given the extremely low number of relevant labels per query, each $S$ matrix is very sparse and can be computed and stored efficiently. 

![pairwise_train_loss](img/pairwise_train_loss.png)

![pairwise_train_mndcg](img/pairwise_train_mndcg.png)

![pairwise_val_mndcg](img/pairwise_val_mndcg.png)




## Listwise LambdaNet

Plots, results, comments

Note that the calculation of $|\Delta NDCG|$ can be simplified from calculating NDCG twice to only considering the discounted gain that each of the swapped documents generates. Let's assume that we have a ranking in which the documents at position $i$ and $j$ are to be interchanged:

$$ R1 = [ 1, 2, 3, \ldots, i, \ldots, j, \ldots, 1000]$$
$$ \, \, \,\, \, \,\, \, \,\, \, \,\, \, \,\,  \uparrow \_\_ \uparrow$$

This generates the ranking:
$$ R2 = [ 1, 2, 3, \ldots, j, \ldots, i, \ldots, 1000]$$

Thus, calculating the $|\Delta NDCG|$ is equivalent to:

\begin{eqnarray}
\Delta NDCG &=& \left|NDCG(R2) - NDCG(R1) \right| \\
&=& \left| \left[ \sum_{k \ne i,j} \frac{2^{rel_k} -1 }{\log(1+k)} + \frac{2^{rel_i} - 1 }{\log(1+j)} + \frac{2^{rel_j} -1 }{\log(1+i)} \right] - \left[ \sum_{k \ne i,j} \frac{2^{rel_k} -1 }{\log(1+k)} + \frac{2^{rel_i} - 1 }{\log(1+i)} + \frac{2^{rel_j} -1 }{\log(1+j)} \right] \right| \\
&=& \left|\left( 2^{rel_i} - 2^{rel_j} \right) \left( \frac{1}{\log(1+j)} - \frac{1}{\log(1+i)}\right) \right|
\end{eqnarray}



![listwise_train_loss](img/listwise_train_loss.png)

![listwise_train_mndcg](img/listwise_train_mndcg.png)

![listwise_val_mndcg](img/listwise_val_mndcg.png)


## Model Comparison

![pointwise_mean_train_val](img/pointwise_mean_train_val.png)
![pairwise_mean_train_val](img/pairwise_mean_train_val.png)
![listwise_mean_train_val](img/listwise_mean_train_val.png)

Statistical tests